In [ ]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the path to your dataset
data_path = 'C:/Users/harsh/Desktop/wav'

# Get the list of genres from the directory names
genres = os.listdir(data_path)

# Initialize lists to hold features and labels
features = []
labels = []

# Iterate over each genre
for genre in genres:
    # Get the list of song files for this genre
    song_files = os.listdir(os.path.join(data_path, genre))
    
    # Iterate over each song file
    for song in song_files:
        # Load the song file using librosa
        y, sr = librosa.load(os.path.join(data_path, genre, song))
        
        # Extract MFCC features from the audio
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Using 13 MFCC features
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        
        # Concatenate features
        features.append(np.concatenate((np.mean(mfcc, axis=1), 
                                         np.mean(spectral_centroid, axis=1), 
                                         np.mean(spectral_bandwidth, axis=1), 
                                         np.mean(spectral_contrast, axis=1))))
        labels.append(genre)

# Convert features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for the LSTM
X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(128, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(BatchNormalization())
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(BatchNormalization())
model_lstm.add(LSTM(64))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(len(genres), activation='softmax'))

# Compile the LSTM model
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the LSTM model
model_lstm.fit(X_train_lstm, y_train, epochs=1023, batch_size=64, validation_data=(X_test_lstm, y_test))

# Use the LSTM model to extract features from the data
X_train_svm = model_lstm.predict(X_train_lstm)
X_test_svm = model_lstm.predict(X_test_lstm)

# Train an SVM on the features extracted by the LSTM
model_svm = SVC()
model_svm.fit(X_train_svm, y_train)

# Make predictions with the SVM
y_pred = model_svm.predict(X_test_svm)

# Calculate the accuracy of the SVM
accuracy = accuracy_score(y_test, y_pred)

print(f"SVM Accuracy: {accuracy}")


In [ ]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the path to your dataset
data_path = 'C:/Users/harsh/Desktop/wav'

# Get the list of genres from the directory names
genres = os.listdir(data_path)

# Initialize lists to hold features and labels
features = []
labels = []

# Iterate over each genre
for genre in genres:
    # Get the list of song files for this genre
    song_files = os.listdir(os.path.join(data_path, genre))
    
    # Iterate over each song file
    for song in song_files:
        # Load the song file using librosa
        y, sr = librosa.load(os.path.join(data_path, genre, song))
        
        # Extract MFCC features from the audio
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Using 13 MFCC features
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        
        # Concatenate features
        features.append(np.concatenate((np.mean(mfcc, axis=1), 
                                         np.mean(spectral_centroid, axis=1), 
                                         np.mean(spectral_bandwidth, axis=1), 
                                         np.mean(spectral_contrast, axis=1))))
        labels.append(genre)

# Convert features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for the LSTM
X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(128, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(BatchNormalization())
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(BatchNormalization())
model_lstm.add(LSTM(64))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(len(genres), activation='softmax'))

# Compile the LSTM model
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the LSTM model
model_lstm.fit(X_train_lstm, y_train, epochs=1023, batch_size=64, validation_data=(X_test_lstm, y_test))

# Use the LSTM model to extract features from the data
X_train_svm = model_lstm.predict(X_train_lstm)
X_test_svm = model_lstm.predict(X_test_lstm)

# Define parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=5)
grid_search.fit(X_train_svm, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train SVM with best hyperparameters
best_svm_clf = SVC(**best_params)
best_svm_clf.fit(X_train_svm, y_train)

# Make predictions with the SVM
y_pred = best_svm_clf.predict(X_test_svm)

# Calculate the accuracy of the SVM
accuracy = accuracy_score(y_test, y_pred)

print(f"SVM Accuracy: {accuracy}")
